## Introduction
## Module 3 Deep Learning Homework Submission
### Peer-graded Assignment: Build a Regression Model in Keras

<a id="item31"></a>

## Download and Clean Dataset

In [17]:
import pandas as pd
import numpy as np

In [18]:
import keras

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [19]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [20]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [21]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [22]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

In [23]:
from keras.models import Sequential
from keras.layers import Dense

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [24]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [25]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [26]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [61]:
n_cols = predictors.shape[1] # number of predictors

<a id="item1"></a>

In [62]:
n_cols

8

<a id='item32'></a>

In [63]:
import keras

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [64]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>

## Part A: HW Submission: Build a baseline model 

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [65]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

<a id="item4"></a>

In [66]:
model = regression_model()

<a id='item34'></a>

In [67]:
from sklearn.model_selection import train_test_split

In [68]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=50)

In [69]:
X_train.shape

(721, 8)

In [70]:
# fit the model
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
721/721 [==============================] - 1s 1ms/step - loss: 97039.8984
Epoch 2/50
721/721 [==============================] - 0s 411us/step - loss: 20177.0520
Epoch 3/50
721/721 [==============================] - 0s 609us/step - loss: 7265.68790s - loss: 7253.51
Epoch 4/50
721/721 [==============================] - 0s 359us/step - loss: 6439.8375
Epoch 5/50
721/721 [==============================] - 0s 386us/step - loss: 5813.0073
Epoch 6/50
721/721 [==============================] - 0s 387us/step - loss: 5238.0157
Epoch 7/50
721/721 [==============================] - 0s 361us/step - loss: 4786.7081
Epoch 8/50
721/721 [==============================] - 0s 411us/step - loss: 4431.1152
Epoch 9/50
721/721 [==============================] - 0s 444us/step - loss: 4123.5565
Epoch 10/50
721/721 [==============================] - 0s 388us/step - loss: 3836.2380
Epoch 11/50
721/721 [==============================] - 0s 438us/step - loss: 3583.7281
Epoch 12/50
721/721 [=============

In [71]:
predictions_test = model.predict(X_test)

In [72]:
from sklearn.metrics import mean_squared_error

In [73]:
loss_value = model.evaluate(X_test, y_test)
y_predict = model.predict(X_test)
loss_value

309/309 [==============================] - 0s 459us/step


346.9261879535169

In [74]:
mean_square_error = mean_squared_error(y_test, y_predict)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)


In [75]:
print(mean, standard_deviation)

346.926190385321 0.0


In [77]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("Mean squared error "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)

mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')


print("mean: "+str(mean))

print("standard deviation of the mean squared errors: "+str(standard_deviation))

Mean squared error 1: 93.01293166324159
Mean squared error 2: 129.97550621156168
Mean squared error 3: 113.50431497814586
Mean squared error 4: 136.25990562068606
Mean squared error 5: 118.92057128708726
Mean squared error 6: 107.18770789482832
Mean squared error 7: 144.15330495494854
Mean squared error 8: 97.2755575828182
Mean squared error 9: 130.11219540691684
Mean squared error 10: 168.74353501403215
Mean squared error 11: 101.3845046947689
Mean squared error 12: 139.94989023548115
Mean squared error 13: 126.77931205971727
Mean squared error 14: 127.93118676244248
Mean squared error 15: 117.71298153577885
Mean squared error 16: 114.33330585269866
Mean squared error 17: 106.87316709351771
Mean squared error 18: 96.21595166647704
Mean squared error 19: 94.07627700756282
Mean squared error 20: 94.40799115622313
Mean squared error 21: 51.71358359130069
Mean squared error 22: 59.78622524175058
Mean squared error 23: 48.043112436930336
Mean squared error 24: 47.50027405328349
Mean square